In [12]:
######################################################
######## Step0. Set up Environment Variables #########
######################################################

### GDC DATA PORTAL: https://gdc.cancer.gov/

import re, os, scipy, json
import numpy as np
from scipy import stats
from os import system
from collections import OrderedDict, defaultdict
from operator import itemgetter
from pprint import pprint
from xml.etree.ElementTree import parse
from datetime import datetime
import string


easyTCGA = '/home/ruidong/Documents/Research/Cancer_Genomics/easyTCGA/'       ###### !!!!!!!!
databases = '/home/ruidong/Documents/Research/Cancer_Genomics/Data/Databases/'

### data type <not in use>
RNAseq,miRNA,isoform,methylation27,methylation450,CNS,maskedCNS,MaskedSM,clinical,biospecimen = \
['Gene_Expression_Quantification/','miRNA_Expression_Quantification/','Isoform_Expression_Quantification/',
 'Methylation_Beta_Value27/','Methylation_Beta_Value450/','Copy_Number_Segment/','Masked_Copy_Number_Segment/',
 'Masked_Somatic_Mutation/','Clinical_Supplement/','Biospecimen_Supplement/']


### working directory
miRNAmRNA = 'miRNA_mRNA_network/'
survival = 'Survival/'

downloads = 'Downloads/'
clean = 'Clean/'
analysis = 'Analysis/'

os.chdir(easyTCGA)

### projects summary, add more information
projects = OrderedDict()
with open('projects.2017-02-03T22-29-28.039066.txt', 'rt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        if line.startswith('ID'):
            continue
        projects[lst[0]] = lst[1]
        

print ('Total Number of projects in GDC portal: ' + str(len(projects)))
print ('###################################################')
for ke, val in projects.items():
    print (ke+'\t'+val)
    
print ('###################################################\n')

genoDataType = int(input('What type is the data?\n\
(1.mRNA 2.miRNA 3.isoform 4.methylation27 5.methylation450 6.copy number 7.masked copy number 8.clinical 9.biospecimen 10.snp):\n'))

Total Number of projects in GDC portal: 39
###################################################
TARGET-NBL	Neuroblastoma
TCGA-BRCA	Breast Invasive Carcinoma
TARGET-AML	Acute Myeloid Leukemia
TARGET-WT	High-Risk Wilms Tumor
TCGA-GBM	Glioblastoma Multiforme
TCGA-OV	Ovarian Serous Cystadenocarcinoma
TCGA-LUAD	Lung Adenocarcinoma
TCGA-UCEC	Uterine Corpus Endometrial Carcinoma
TCGA-KIRC	Kidney Renal Clear Cell Carcinoma
TCGA-HNSC	Head and Neck Squamous Cell Carcinoma
TCGA-LGG	Brain Lower Grade Glioma
TCGA-THCA	Thyroid Carcinoma
TCGA-LUSC	Lung Squamous Cell Carcinoma
TCGA-PRAD	Prostate Adenocarcinoma
TCGA-STAD	Stomach Adenocarcinoma
TCGA-SKCM	Skin Cutaneous Melanoma
TCGA-COAD	Colon Adenocarcinoma
TCGA-BLCA	Bladder Urothelial Carcinoma
TARGET-OS	Osteosarcoma
TCGA-LIHC	Liver Hepatocellular Carcinoma
TCGA-CESC	Cervical Squamous Cell Carcinoma and Endocervical Adenocarcinoma
TCGA-KIRP	Kidney Renal Papillary Cell Carcinoma
TCGA-SARC	Sarcoma
TCGA-LAML	Acute Myeloid Leukemia
TCGA-PAAD	Pancreatic Ade

In [44]:
###################################################
############# Data organization ###################
###################################################

############### targets #########################

### miRNA, isoform, cnv, RNAseq


uuid2sample = OrderedDict()

metaProjects = OrderedDict()
uuid = OrderedDict()
samples = OrderedDict()
traits = OrderedDict()

dataType = maskedCNS
samType = 'masked_cnv'


with open(easyTCGA+samType+'.metadata.cart.2017-02-06.all.basic_information.txt', 'rt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        
        lst = [trt.replace(' ','') for trt in lst]
        if line.startswith('file_name'):
            traitName = lst[4:16]
            #fh.write('\t'+'\t'.join(lst[4:16])+'\n')
            continue
            
        elif lst[-1] not in metaProjects.keys():
            metaProjects[lst[-1]] = []
            
        metaProjects[lst[-1]].append(lst[3]+'\t'+'\t'.join(lst[4:16]))
            

for ke, val in metaProjects.items():
    fh = open(ke+'/'+dataType+analysis+samType+'.targets.'+ke+'.txt', 'wt')
    fh.write('\t'+'\t'.join(traitName)+'\n')
    fh.write('\n'.join(sorted(val)))
    
    fh.close()
    


    
############# clean ################
for ke in metaProjects.keys():
    fh = open(ke+'/'+dataType+analysis+samType+'.targets.clean.'+ke+'.txt', 'wt')
    
    checkSam = OrderedDict()
    samples = []
    with open(ke+'/'+dataType+analysis+samType+'.targets.'+ke+'.txt') as f:
        for line in f:
            line = line.rstrip()
            lst = line.split('\t')
            
            if lst[0] == '':
                fh.write(line+'\n')
                continue
                
            else:
                samples.append(lst[0])
                checkSam[lst[0]] = line
                
    uniSam = OrderedDict()
    
    for sam in samples:
        for x in string.ascii_uppercase:
            xSam = re.sub('[A-Z]$', x, sam)

            if xSam in samples:
                uniSam[xSam] = 1
                break

    for sam in sorted(uniSam.keys()):
        fh.write(checkSam[sam]+'\n')
    
    fh.close()

In [13]:
######################################################
############# Extract mature miRNA ###################
######################################################

mirbase21 = OrderedDict()

with open(databases+'miRBase/mature.release21.id.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        mirbase21[lst[0]] = lst[1]


for prjt in projects.keys():
    
    if not os.path.isdir(easyTCGA+prjt+'/'+isoform):
        continue
    
    os.chdir(easyTCGA+prjt+'/'+isoform)
    files = os.listdir(clean)

    for fl in files:
        if fl.endswith('.txt') and not fl.startswith('mature'):
            fh = open(clean+'mature.counts.'+fl, 'wt')
        
            mir = OrderedDict()
            with open(clean+fl) as f:
                for line in f:
                    line = line.rstrip()
        
                    lst = line.split('\t')
        
                    if not lst[-1].startswith('mature'):
                        continue
            
                    else:
                        mimat = lst[-1].split(',')[-1]
            
                        if mimat not in mir.keys():
                            mir[mimat] = 0
                        mir[mimat] += float(lst[2])
            
            
            for ke in mirbase21.keys():
                if ke not in mir.keys():
                    fh.write(mirbase21[ke]+'\t0\n')
        
                else:
                    fh.write(mirbase21[ke]+'\t'+str(mir[ke])+'\n')
    
    
            fh.close()


In [15]:
####################################################
############# Merge mature miRNA ###################
####################################################

for prjt in projects:
    
    if not os.path.isdir(easyTCGA+prjt+'/'+isoform):
        continue
    
    samples = []
    uuid2sample = OrderedDict()
    with open(easyTCGA+'isoform.metadata.cart.2017-02-06.all.basic_information.txt') as f:
        for line in f:
            line = line.rstrip()
        
            lst = line.split('\t')
        
            if lst[-1] == prjt:
                samples.append(lst[3])
                uuid2sample[lst[1]] = lst[3]
                
    
    uniSam = OrderedDict()
    for sam in samples:
        for x in string.ascii_uppercase:
            xSam = re.sub('[A-Z]$', x, sam)

            if xSam in samples:
                uniSam[xSam] = 1
                break
    
    
    
    os.chdir(easyTCGA+prjt+'/'+isoform)
    files = sorted(os.listdir(clean))
    
    genoSamples = []
    genotype = OrderedDict()
    
    for fl in files:
        if fl.startswith('mature.counts'):
            flName = fl.split('.')[-2]
            genoSamples.append(uuid2sample[flName])
        
            with open(clean+fl, 'rt') as f:
                for line in f:
                    if re.match('miRNA_ID|sample|Composite', line):
                        continue
                    line = line.rstrip()
                    lst = line.split('\t')
                
                    if lst[0] not in genotype.keys():
                        genotype[lst[0]] = []
                    genotype[lst[0]].append(lst[1])
    
    
    
    ##### all samples
    fh = open(analysis+'mature_miRNAs_expression.counts.'+prjt+'.txt', 'wt')

    fh.write('\t'+'\t'.join(genoSamples)+'\n')
    
    for ke, val in genotype.items():
        fh.write(ke+'\t'+'\t'.join(val)+'\n')

    fh.close()
    
    
    ##### clean
    fh = open(analysis+'mature_miRNAs_expression.counts.clean.'+prjt+'.txt', 'wt')

    samples = sorted(uniSam.keys())

    sampleIndex = [genoSamples.index(sam) for sam in samples]                               
    fh.write('\t'+'\t'.join([genoSamples[i] for i in sampleIndex])+'\n')

    for ke, val in genotype.items():
        fh.write(ke+'\t'+'\t'.join([val[i] for i in sampleIndex])+'\n')

    fh.close()
    

In [57]:
###################################################
############# Merge RNAseq data ###################
###################################################


for prjt in projects.keys():
    
    if not os.path.isdir(easyTCGA+prjt+'/'+RNAseq):
        continue
    
    samples = []
    uuid2sample = OrderedDict()
    with open(easyTCGA+'RNAseq.metadata.cart.2017-02-03.all.basic_information.txt') as f:
        for line in f:
            line = line.rstrip()
        
            lst = line.split('\t')
        
            if lst[-1] == prjt:
                samples.append(lst[3])
                uuid2sample[lst[1]] = lst[3]
                
    
    uniSam = OrderedDict()
    for sam in samples:
        for x in string.ascii_uppercase:
            xSam = re.sub('[A-Z]$', x, sam)

            if xSam in samples:
                uniSam[xSam] = 1
                break
    
    
    
    os.chdir(easyTCGA+prjt+'/'+RNAseq)
    files = sorted(os.listdir(clean))
    
    genoSamples = []
    genotype = OrderedDict()
    
    for fl in files:
        flName = fl.split('.')[0]
        genoSamples.append(uuid2sample[flName])
        
        with open(clean+fl, 'rt') as f:
            for line in f:
                if re.match('miRNA_ID|sample|Composite|__', line):
                    continue
                line = line.rstrip()
                lst = line.split('\t')
                
                if lst[0] not in genotype.keys():
                    genotype[lst[0]] = []
                genotype[lst[0]].append(lst[1])
    
    
    
    ##### all samples
    fh = open(analysis+'RNAseq.raw_counts.'+prjt+'.txt', 'wt')

    fh.write('\t'+'\t'.join(genoSamples)+'\n')
    
    for ke, val in genotype.items():
        fh.write(ke+'\t'+'\t'.join(val)+'\n')

    fh.close()
    
    print ('Merge RNAseq data from project %s is completed !' %prjt)
    
    
    ##### clean
    fh = open(analysis+'RNAseq.raw_counts.clean.'+prjt+'.txt', 'wt')

    samples = sorted(uniSam.keys())

    sampleIndex = [genoSamples.index(sam) for sam in samples]                               
    fh.write('\t'+'\t'.join([genoSamples[i] for i in sampleIndex])+'\n')

    for ke, val in genotype.items():
        fh.write(ke+'\t'+'\t'.join([val[i] for i in sampleIndex])+'\n')

    fh.close()
    
    print ('Clean RNAseq data from project %s is completed !' %prjt)


Merge RNAseq data from project TCGA-BRCA is completed !
Clean RNAseq data from project TCGA-BRCA is completed !
Merge RNAseq data from project TCGA-GBM is completed !
Clean RNAseq data from project TCGA-GBM is completed !
Merge RNAseq data from project TCGA-OV is completed !
Clean RNAseq data from project TCGA-OV is completed !
Merge RNAseq data from project TCGA-LUAD is completed !
Clean RNAseq data from project TCGA-LUAD is completed !
Merge RNAseq data from project TCGA-UCEC is completed !
Clean RNAseq data from project TCGA-UCEC is completed !
Merge RNAseq data from project TCGA-KIRC is completed !
Clean RNAseq data from project TCGA-KIRC is completed !
Merge RNAseq data from project TCGA-HNSC is completed !
Clean RNAseq data from project TCGA-HNSC is completed !
Merge RNAseq data from project TCGA-LGG is completed !
Clean RNAseq data from project TCGA-LGG is completed !
Merge RNAseq data from project TCGA-THCA is completed !
Clean RNAseq data from project TCGA-THCA is completed !


In [60]:
###################################################
############# Merge miRNA data ####################
###################################################


for prjt in projects.keys():
    
    if not os.path.isdir(easyTCGA+prjt+'/'+miRNA):
        continue
    
    samples = []
    uuid2sample = OrderedDict()
    with open(easyTCGA+'miRseq.metadata.cart.2017-02-06.all.basic_information.txt') as f:
        for line in f:
            line = line.rstrip()
        
            lst = line.split('\t')
        
            if lst[-1] == prjt:
                samples.append(lst[3])
                uuid2sample[lst[1]] = lst[3]
                
    
    uniSam = OrderedDict()
    for sam in samples:
        for x in string.ascii_uppercase:
            xSam = re.sub('[A-Z]$', x, sam)

            if xSam in samples:
                uniSam[xSam] = 1
                break
    
    
    
    os.chdir(easyTCGA+prjt+'/'+miRNA)
    files = sorted(os.listdir(clean))
    
    genoSamples = []
    genotype = OrderedDict()
    
    for fl in files:
        flName = fl.split('.')[0]
        #if flName not in uuid2sample.keys():
        #    print (flName)
        #    continue
        genoSamples.append(uuid2sample[flName])
        
        with open(clean+fl, 'rt') as f:
            for line in f:
                if re.match('miRNA_ID|sample|Composite|__', line):
                    continue
                line = line.rstrip()
                lst = line.split('\t')
                
                if lst[0] not in genotype.keys():
                    genotype[lst[0]] = []
                genotype[lst[0]].append(lst[1])  ### counts:lst[1], rpm:lst[2]
    
    
    
    ##### all samples
    fh = open(analysis+'miRNAseq.raw_counts.'+prjt+'.txt', 'wt')

    fh.write('\t'+'\t'.join(genoSamples)+'\n')
    
    for ke, val in genotype.items():
        fh.write(ke+'\t'+'\t'.join(val)+'\n')

    fh.close()
    
    print ('Merge miRNAseq data from project %s is completed !' %prjt)
    
    
    ##### clean
    fh = open(analysis+'miRNAseq.raw_counts.clean.'+prjt+'.txt', 'wt')

    samples = sorted(uniSam.keys())

    sampleIndex = [genoSamples.index(sam) for sam in samples]                               
    fh.write('\t'+'\t'.join([genoSamples[i] for i in sampleIndex])+'\n')

    for ke, val in genotype.items():
        fh.write(ke+'\t'+'\t'.join([val[i] for i in sampleIndex])+'\n')

    fh.close()
    
    print ('Clean miRNAseq data from project %s is completed !' %prjt)


Merge miRNAseq data from project TCGA-BRCA is completed !
Clean miRNAseq data from project TCGA-BRCA is completed !
Merge miRNAseq data from project TCGA-GBM is completed !
Clean miRNAseq data from project TCGA-GBM is completed !
Merge miRNAseq data from project TCGA-OV is completed !
Clean miRNAseq data from project TCGA-OV is completed !
Merge miRNAseq data from project TCGA-LUAD is completed !
Clean miRNAseq data from project TCGA-LUAD is completed !
Merge miRNAseq data from project TCGA-UCEC is completed !
Clean miRNAseq data from project TCGA-UCEC is completed !
Merge miRNAseq data from project TCGA-KIRC is completed !
Clean miRNAseq data from project TCGA-KIRC is completed !
Merge miRNAseq data from project TCGA-HNSC is completed !
Clean miRNAseq data from project TCGA-HNSC is completed !
Merge miRNAseq data from project TCGA-LGG is completed !
Clean miRNAseq data from project TCGA-LGG is completed !
Merge miRNAseq data from project TCGA-THCA is completed !
Clean miRNAseq data fr

In [ ]:
###################################################
############# Merge methylation data ##############
###################################################


for prjt in projects.keys():
    
    if not os.path.isdir(easyTCGA+prjt+'/'+methylation450):
        continue
    
    samples = []
    uuid2sample = OrderedDict()
    with open(easyTCGA+'miRseq.metadata.cart.2017-02-06.all.basic_information.txt') as f:
        for line in f:
            line = line.rstrip()
        
            lst = line.split('\t')
        
            if lst[-1] == prjt:
                samples.append(lst[3])
                uuid2sample[lst[1]] = lst[3]
                
    
    uniSam = OrderedDict()
    for sam in samples:
        for x in string.ascii_uppercase:
            xSam = re.sub('[A-Z]$', x, sam)

            if xSam in samples:
                uniSam[xSam] = 1
                break
    
    
    
    os.chdir(easyTCGA+prjt+'/'+methylation450)
    files = sorted(os.listdir(clean))
    
    genoSamples = []
    genotype = OrderedDict()
    
    for fl in files:
        flName = fl.split('.')[0]
        #if flName not in uuid2sample.keys():
        #    print (flName)
        #    continue
        genoSamples.append(uuid2sample[flName])
        
        with open(clean+fl, 'rt') as f:
            for line in f:
                if re.match('miRNA_ID|sample|Composite|__', line):
                    continue
                line = line.rstrip()
                lst = line.split('\t')
                
                if lst[0] not in genotype.keys():
                    genotype[lst[0]] = []
                genotype[lst[0]].append(lst[1])  ### counts:lst[1], rpm:lst[2]
    
    
    
    ##### all samples
    fh = open(analysis+'methylation450.beta_value.'+prjt+'.txt', 'wt')

    fh.write('\t'+'\t'.join(genoSamples)+'\n')
    
    for ke, val in genotype.items():
        fh.write(ke+'\t'+'\t'.join(val)+'\n')

    fh.close()
    
    print ('Merge methylation450 data from project %s is completed !' %prjt)
    
    
    ##### clean
    fh = open(analysis+'methylation450.beta_value.clean.'+prjt+'.txt', 'wt')

    samples = sorted(uniSam.keys())

    sampleIndex = [genoSamples.index(sam) for sam in samples]                               
    fh.write('\t'+'\t'.join([genoSamples[i] for i in sampleIndex])+'\n')

    for ke, val in genotype.items():
        fh.write(ke+'\t'+'\t'.join([val[i] for i in sampleIndex])+'\n')

    fh.close()
    
    print ('Clean methylation450 data from project %s is completed !' %prjt)


In [79]:
#######################################
#### Step1. Parse Simple Metadata #####
#######################################


def parseAnnotations(metadatai):
    annotationInfo = ['notes', 'category', 'status']
    
    if 'annotations' not in metadatai.keys():
        notes, category, status = None, None, None
        annotations = [notes, category, status]
    else:
        annoNum = len(metadatai['annotations'])
        notes, category, status = [None]*annoNum, [None]*annoNum, [None]*annoNum
        for j in range(annoNum):
            [notes[j], category[j], status[j]] = [metadatai['annotations'][j][anno] for anno in annotationInfo]
        annotations = [';'.join(map(str,notes)), ';'.join(map(str,category)), ';'.join(map(str,status))]
        annotations = [re.sub('\r+|\n+','',anno) for anno in annotations]
    return annotations


def tcgaMetaParser(metaFile, genoDataType):
    
    sampleInfo = ['submitter_id', 'tissue_type', 'sample_type', 'tumor_code_id', 'sample_type_id']
    basicDemographic = ['race', 'gender', 'ethnicity']
    basicDiagnoses = ['tumor_stage','tumor_grade','age_at_diagnosis','days_to_death','days_to_last_known_disease_status',
                      'days_to_last_follow_up','vital_status']
    annotationInfo = ['notes', 'category', 'status']
    
    fhTmp = open(metaFile+'.basic_information.tmp', 'wt')

    
    with open(metaFile+'.json', 'rt') as f:
        metadata = json.load(f)
        
    for i in range(len(metadata)):
 
        fileName = metadata[i]['file_name']
        fileId = metadata[i]['file_id']
        dataType = metadata[i]['data_type'].replace(' ', '_')

        annotations = parseAnnotations(metadata[i])
        
        for k in metadata[i].keys():
            if k == 'cases':
                caseId = metadata[i][k][0]['case_id']
                projectId = metadata[i][k][0]['project']['project_id']
                
                
                if genoDataType in [7,8]:
                    [submitterId, tissueType, sampleType, tumorCodeId, sampleTypeId] = [None]*5
                    submitterId = fileName.split('.')[-2]
                else:
                    [submitterId, tissueType, sampleType, tumorCodeId, sampleTypeId] = \
                    [metadata[i][k][0]['samples'][0][sam] for sam in sampleInfo]

                if 'diagnoses' not in metadata[i][k][0].keys():
                    numBlank = len(basicDemographic)+len(basicDiagnoses)
                    
                    fhTmp.write('\t'.join(map(str,[fileName,fileId,caseId,submitterId,sampleType,sampleTypeId]\
                                                +[None]*numBlank+annotations+[dataType,projectId]))+'\n')
                    
                else:
                    basicDemo = [race, gender, ethnicity]\
                              = [metadata[i][k][0]['demographic'][demo] for demo in basicDemographic]
                
                    basicDiag = [tumorStage,tumorGrade,ageAtDiagnosis,daysToDeath,daysToLastKnownStatus,daysToLastFollowUp,
                             vitalStatus]=[metadata[i][k][0]['diagnoses'][0][diag] for diag in basicDiagnoses]
                

                    fhTmp.write('\t'.join(map(str, [fileName,fileId,caseId,submitterId,sampleType,sampleTypeId]\
                                    +basicDiag+basicDemo+annotations+[dataType,projectId]))+'\n')
        
    fhTmp.close()
    
    fh = open(metaFile+'.basic_information.txt', 'wt')
    fh.write('\t'.join(map(str, ['file_name','file_uuid','case_id','submitter_id','sample_type','sample_type_id']\
            +basicDiagnoses+basicDemographic+annotationInfo+['data_type','project_id']))+'\n')

    newFile = OrderedDict()
    with open(metaFile+'.basic_information.tmp', 'rt') as f:
        for line in f:
            line = line.rstrip()
            lst = line.split('\t')
            if lst[-1] not in newFile.keys():
                newFile[lst[-1]] = OrderedDict()
            newFile[lst[-1]][lst[1]] = line
    
    print ()
    print ('################################################################')
    
    for ke,val in newFile.items():
        print ('Number of %s samples in project %s: %d' %(dataFolders[genoDataType],ke,len(val)))
        
        for subk, subv in val.items():
            subv = re.sub(r'not reported|None', 'NA', subv)
            fh.write(subv+'\n')
    
    fh.close()

    os.system('rm '+metaFile+'.basic_information.tmp')

#########################################################################
metaFile = input('Metadata file name (no .json extension):\n')
tcgaMetaParser(metaFile,genoDataType)

Metadata file name (no .json extension):
clinical.metadata.cart.2017-02-05.all

################################################################
Number of Clinical_Supplement samples in project TCGA-LUAD: 522
Number of Clinical_Supplement samples in project TCGA-GBM: 596
Number of Clinical_Supplement samples in project TCGA-LGG: 515
Number of Clinical_Supplement samples in project TCGA-OV: 587
Number of Clinical_Supplement samples in project TCGA-LUSC: 504
Number of Clinical_Supplement samples in project TCGA-PRAD: 500
Number of Clinical_Supplement samples in project TCGA-BRCA: 1097
Number of Clinical_Supplement samples in project TCGA-BLCA: 412
Number of Clinical_Supplement samples in project TCGA-PCPG: 179
Number of Clinical_Supplement samples in project TCGA-MESO: 87
Number of Clinical_Supplement samples in project TCGA-THYM: 124
Number of Clinical_Supplement samples in project TCGA-LIHC: 377
Number of Clinical_Supplement samples in project TCGA-SARC: 261
Number of Clinical_Suppleme

In [5]:
####################################################
######### Step2. Summarize metadata files ##########
####################################################

try:
    metaFile
except NameError:
    metaFile = input('Metadata file name (no .json extension):\n')

# del metaFile ### can be used to remove a variable

metaProjects = OrderedDict()
uuid = OrderedDict()
samples = OrderedDict()
traits = OrderedDict()

with open(easyTCGA+metaFile+'.basic_information.txt', 'rt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        if line.startswith('file_name'):
            traitName = lst[4:16]
            continue
            
        elif lst[-1] not in metaProjects.keys():
            metaProjects[lst[-1]] = 0
            uuid[lst[-1]] = OrderedDict()
            traits[lst[-1]] = OrderedDict()
            samples[lst[-1]] = []
            
        metaProjects[lst[-1]] += 1
        uuid[lst[-1]][lst[1]] = lst[3]
        samples[lst[-1]].append(lst[3])
        
        for i in range(len(traitName)):
            trt = traitName[i]
            if trt not in traits[lst[-1]].keys():
                traits[lst[-1]][trt] = []
            traits[lst[-1]][trt].append(lst[i+4].replace(' ',''))
            
            
print ('Number of projects: %d' %len(metaProjects))
print ('Total Number of samples: %d' %sum([val for val in metaProjects.values()]))
print ('#################################################')

for ke, val in metaProjects.items():
    print ('Number of samples in project %s: %d' %(ke,val))
    

Metadata file name (no .json extension):
clinical.metadata.cart.2017-02-05.all
Number of projects: 33
Total Number of samples: 11160
#################################################
Number of samples in project TCGA-LUAD: 522
Number of samples in project TCGA-GBM: 596
Number of samples in project TCGA-LGG: 515
Number of samples in project TCGA-OV: 587
Number of samples in project TCGA-LUSC: 504
Number of samples in project TCGA-PRAD: 500
Number of samples in project TCGA-BRCA: 1097
Number of samples in project TCGA-BLCA: 412
Number of samples in project TCGA-PCPG: 179
Number of samples in project TCGA-MESO: 87
Number of samples in project TCGA-THYM: 124
Number of samples in project TCGA-LIHC: 377
Number of samples in project TCGA-SARC: 261
Number of samples in project TCGA-PAAD: 185
Number of samples in project TCGA-KIRC: 537
Number of samples in project TCGA-STAD: 443
Number of samples in project TCGA-CESC: 307
Number of samples in project TCGA-ACC: 92
Number of samples in project TC

In [8]:
#############################################
##### Step3. Create required directories ####
#############################################

### if exits, do nothing
dataType = ['Gene_Expression_Quantification',
            'miRNA_Expression_Quantification',
            'Isoform_Expression_Quantification',
            'Methylation_Beta_Value27',
            'Methylation_Beta_Value450',
            'Copy_Number_Segment',
            'Masked_Copy_Number_Segment',
            'Clinical_Supplement',
            'Biospecimen_Supplement',
            'Raw Simple Somatic Mutation',
            'Masked_Somatic_Mutation']

genoFile = ['RNAseq.counts',
            'miRseq.counts',
            'isoform.counts',
            'methylation.beta_value27',
            'methylation.beta_value450',
            None,
            None,
            'Merged_clinical_data.',
            None,
            None]

dataFolders = OrderedDict()
for i in range(1,11):
    dataFolders[i] = dataType[i-1]
    
genoDataNames = OrderedDict()
for i in range(1,11):
    genoDataNames[i] = genoFile[i-1]
    
newFolders = ' '.join([easyTCGA+prjt+'/'+dataFolders[genoDataType]+'/'+fd for prjt in metaProjects 
                       for fd in ['Downloads','Clean','Analysis']])
os.system('mkdir -p '+newFolders)

0

In [ ]:
#########################################
#### Step4. Download and clean data #####
#########################################


def downloadTCGA(project):
    
    os.chdir(easyTCGA+'/'.join([project,dataFolders[genoDataType],downloads]))
   
    print ('##############################################################')
    
    print (project+'\tStart Downloading ... (%s)' %datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
    fileUUID = ' '.join(uuid[project])
    os.system('gdc-client download '+fileUUID)
    print (project+'\tDownloading completed ! (%s)' %datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
    #numSamples = len([fd for fd in os.listdir('.') if os.path.isdir(fd)]) ###count number of folders
    print ('Number of downloaded %s samples: %d' %(dataFolders[genoDataType],len(os.listdir('.'))))

### all file names would be changed to the corresponding folder names (uuids)
def cleanTCGA(project):
    os.chdir(easyTCGA+project+'/'+dataFolders[genoDataType])
    
    print (project+'\tStart Cleaning ...')
    
    folders = os.listdir(downloads)
    if genoDataType == 1:
        for fd in folders:
            if os.path.isdir(downloads+fd):
                os.system('cp '+downloads+fd+'/*.gz '+clean+fd+'.gz')
        os.system('gunzip '+clean+'*.gz')
    
    elif genoDataType in [2,3]:
        for fd in folders:
            if os.path.isdir(downloads+fd):
                os.system('cp '+downloads+fd+'/*.quantification.txt '+clean+fd+'.txt')
             
    elif genoDataType in [4,5]:
        for fd in folders:
            if os.path.isdir(downloads+fd):
                os.system('cp '+downloads+fd+'/*.gdc_hg38.txt '+clean+fd+'.txt')

    if genoDataType in [8,9]:
        for fd in folders:
            if os.path.isdir(downloads+fd):
                os.system('cp '+downloads+fd+'/*.xml '+clean+fd+'.xml')
    
    print (project+'\tCleaning completed !')
    print ('Number of cleaned %s samples: %d' %(dataFolders[genoDataType],len(os.listdir(clean))))
    
                
        
############################################################
for project in metaProjects.keys():
    downloadTCGA(project)
    cleanTCGA(project)

TCGA-LUAD	Start Downloading ... (2017-02-05 13:57:17)
TCGA-LUAD	Downloading completed ! (2017-02-05 14:24:50)
Number of downloaded Clinical_Supplement samples: 522
TCGA-LUAD	Start Cleaning ...
TCGA-LUAD	Cleaning completed !
Number of cleaned Clinical_Supplement samples: 1044
################################################
TCGA-GBM	Start Downloading ... (2017-02-05 14:24:52)
TCGA-GBM	Downloading completed ! (2017-02-05 14:48:19)
Number of downloaded Clinical_Supplement samples: 596
TCGA-GBM	Start Cleaning ...
TCGA-GBM	Cleaning completed !
Number of cleaned Clinical_Supplement samples: 596
################################################
TCGA-LGG	Start Downloading ... (2017-02-05 14:48:21)
TCGA-LGG	Downloading completed ! (2017-02-05 15:27:32)
Number of downloaded Clinical_Supplement samples: 515
TCGA-LGG	Start Cleaning ...
TCGA-LGG	Cleaning completed !
Number of cleaned Clinical_Supplement samples: 515
################################################
TCGA-OV	Start Downloading ... (2017

In [64]:
##########################################################
#### Step5. Merge Genotyping Data Files into One File ####
#########################################################

def mergeGenotype(project):
    os.chdir(easyTCGA+project+'/'+dataFolders[genoDataType])
    
    files = sorted(os.listdir(clean))
    genoSamples = []
    genotype = OrderedDict()
    
    for fl in files:
        flName = fl.split('.')[0]
        genoSamples.append(uuid[project][flName])
        
        with open(clean+fl, 'rt') as f:
            for line in f:
                if re.match('miRNA_ID|sample|Composite', line):
                    continue
                line = line.rstrip()
                lst = line.split('\t')
                
                if lst[0] not in genotype.keys():
                    genotype[lst[0]] = []
                genotype[lst[0]].append(lst[1])
                
    fh = open(analysis+genoDataNames[genoDataType]+'_only.'+project+'.txt', 'wt')
    
    sampleIndex = [genoSamples.index(sam) for sam in samples[project]]
                                
    fh.write('\t'+'\t'.join([genoSamples[i] for i in sampleIndex])+'\n')
    
    for ke, val in genotype.items():
        fh.write(ke+'\t'+'\t'.join([val[i] for i in sampleIndex])+'\n')

    fh.close()
    
    print ('Merge %s data from project %s is completed !' %(dataFolders[genoDataType], project))
    
    
    
##################################################
for project in metaProjects.keys():
        mergeGenotype(project)

Merge miRNA_Expression_Quantification data from project TCGA-PRAD is completed !
Merge miRNA_Expression_Quantification data from project TCGA-BLCA is completed !
Merge miRNA_Expression_Quantification data from project TCGA-LGG is completed !


In [66]:
################################################################
########## Step6. Prepare input for downstream analysis ########
################################################################

def limmaInput(project):
    os.chdir(easyTCGA+project+'/'+dataFolders[genoDataType])
    
    fh = open(analysis+genoDataNames[genoDataType]+'_and_traits.'+project+'.txt', 'wt')

    with open(analysis+genoDataNames[genoDataType]+'_only.'+project+'.txt', 'rt') as f:
        for line in f:
            line = line.rstrip()
            lst = line.split('\t')
            if lst[0] == '':
                fh.write(line+'\n')
                
                for ke, val in traits[project].items():
                    fh.write(ke+'\t'+'\t'.join(map(str,val))+'\n')
            else:
                fh.write(line+'\n')
    fh.close()
    
    print ('Prepare %s data from project %s is completed !' %(dataFolders[genoDataType], project))

    
    
########################################################
for project in metaProjects.keys():
    limmaInput(project)

Prepare miRNA_Expression_Quantification data from project TCGA-PRAD is completed !
Prepare miRNA_Expression_Quantification data from project TCGA-BLCA is completed !
Prepare miRNA_Expression_Quantification data from project TCGA-LGG is completed !


In [ ]:
###################################################
########## Step7. Parse Clinical data #############
###################################################

################# ALSO CAPABALE OF PARSING BIOSPECIMEN DATA, BUT NOT NECESSARY


def parseClinical(project):
    os.chdir(easyTCGA+project+'/'+dataFolders[genoDataType])
    
    fh = open(analysis+genoDataNames[genoDataType]+project+'.txt', 'wt')
    
    xmlFiles = [fl for fl in os.listdir(clean) if fl.endswith('xml')]
    numSamples = len(xmlFiles)
    
    i=-1
    clinicalData = OrderedDict()
    for fl in xmlFiles:
        i += 1
        flName = fl.split('.')[0]
        genoSamples.append(uuid[project][flName])
        
        doc = parse(clean+fl)
        overlapTraits = []  ### check if the trait name has been used
        for elem in doc.iter():
            if elem.text == None:
                elem.text = 'NA'
            elif elem.text.startswith('\n'):
                continue
            else:
                trt = elem.tag.split('}')[1]
                if overlapTraits.count(trt) != 0:
                    trait = trt + '_' + str(overlapTraits.count(trt)+1)
                else:
                    trait = trt

                overlapTraits.append(trt)
                
                if trait not in clinicalData.keys():
                    clinicalData[trait] = ['NA']*numSamples
                clinicalData[trait][i] = elem.text
        
    sampleIndex = [genoSamples.index(sam) for sam in samples[project]]
    
    fh.write('\t'+'\t'.join([genoSamples[i] for i in sampleIndex])+'\n')
    
    for ke, val in clinicalData.items():
        fh.write(ke+'\t'+'\t'.join([val[i] for i in sampleIndex])+'\n')

    fh.close()

    
    
###############################################
for project in metaProjects.keys():
    parseClinical(project)

In [69]:
################################################################
############# Step0. Merge two metadata files ##################

#### Method1, works, the same format as expected
metaFile_1 = input('Name of the first metadata file:\n')
metaFile_2 = input('Name of the second metadata file:\n')

metaFile_new = re.sub('T\d+\S+', '.all', metaFile_1)

fh = open(metaFile_new+'.json', 'wt')
with open(metaFile_1+'.json', 'rt') as f:
    for line in f:
        line = line.rstrip()
        if line == '[':
            lastLine = line
            
        elif line == ']':
            fh.write(lastLine+',\n')
        else:
            fh.write(lastLine+'\n')
            lastLine = line
            
with open(metaFile_2+'.json', 'rt') as f:
    for line in f:
        line = line.rstrip()
        if line == '[':
            continue
        fh.write(line+'\n')

fh.close()

'''
#### Method2, works, but the json file is a one-line file
multiJSON = []
with open('RNAseq.metadata.cart.2017-02-03T22-17-14.782610.breast.lung.brain.bladder.prostate.ovary.json', 'rt') as f:
    multiJSON.extend(json.load(f))
    
with open('RNAseq.metadata.cart.2017-02-03T22-23-54.182001.others.json', 'rt') as f:
    multiJSON.extend(json.load(f))

len(multiJSON)


with open("RNAseq222.TCGA.metadata.cart.2017-02-03.all.json", "wt") as fh:
    json.dump(multiJSON, fh)
    
    
#### Method3, doesn't work, need to manual check the connection line of the two files
os.system('cat RNAseq.metadata.cart.2017-02-03T22-17-14.782610.breast.lung.brain.bladder.prostate.ovary.json \
          RNAseq.metadata.cart.2017-02-03T22-23-54.182001.others.json \
          > RNAseq333.TCGA.metadata.cart.2017-02-03.all.json')
'''

Name of the first metadata file:
clinical.metadata.cart.2017-02-05T20-45-14.793906.1
Name of the second metadata file:
clinical.metadata.cart.2017-02-05T20-37-19.445296.2


'\n#### Method2, works, but the json file is a one-line file\nmultiJSON = []\nwith open(\'RNAseq.metadata.cart.2017-02-03T22-17-14.782610.breast.lung.brain.bladder.prostate.ovary.json\', \'rt\') as f:\n    multiJSON.extend(json.load(f))\n    \nwith open(\'RNAseq.metadata.cart.2017-02-03T22-23-54.182001.others.json\', \'rt\') as f:\n    multiJSON.extend(json.load(f))\n\nlen(multiJSON)\n\n\nwith open("RNAseq222.TCGA.metadata.cart.2017-02-03.all.json", "wt") as fh:\n    json.dump(multiJSON, fh)\n    \n    \n#### Method3, doesn\'t work, need to manual check the connection line of the two files\nos.system(\'cat RNAseq.metadata.cart.2017-02-03T22-17-14.782610.breast.lung.brain.bladder.prostate.ovary.json           RNAseq.metadata.cart.2017-02-03T22-23-54.182001.others.json           > RNAseq333.TCGA.metadata.cart.2017-02-03.all.json\')\n'

In [ ]:
#######################################
########### ONLY USE FOR ONCE !!!

###############################
### Create all directories ####
###############################
import os
os.chdir('/home/ruidong/Documents/Research/Cancer_Genomics/easyTCGA/')

dataType = ['Gene_Expression_Quantification',
            'Copy_Number_Segment',
            'Masked_Copy_Number_Segment',
            'Methylation_Beta_Value',
            'Isoform_Expression_Quantification',
            'miRNA_Expression_Quantification',
            'Biospecimen_Supplement',
            'Clinical_Supplement',
            'Masked_Somatic_Mutation']


with open('projects.2017-02-03T22-29-28.039066.txt', 'rt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        if line.startswith('ID'):
            continue
        
        folders = ' '.join([lst[0]+'/'+datp+'/'+fd for datp in dataType for fd in ['Downloads','Clean','Analysis']])
        os.system('mkdir -p '+folders)

In [32]:
################################################################
########## Step6. Prepare input for downstream analysis ######## FOR BACKUP
################################################################

### sample order is different from the orignial merged genotype file

def limmaInput(project):
    os.chdir(easyTCGA+project+'/'+dataFolders[genoDataType])
    
    fh = open(analysis+genoDataNames[genoDataType]+'_and_traits.'+project+'.txt', 'wt')

    fh.write('\t'+'\t'.join(samples[project].keys())+'\n')

    for ke, val in traits[project].items():
        fh.write(ke+'\t'+'\t'.join(map(str,val))+'\n')


    ### SHOULD CHANGE THE NAME FOR OTHER TYPES OF DATA
    with open(analysis+genoDataNames[genoDataType]+'_only.'+project+'.txt', 'rt') as f:
        for line in f:
            line = line.rstrip()
            lst = line.split('\t')
            if lst[0] == '':
                allSamples = lst[1:]
                sampleIndex = [allSamples.index(sam) for sam in samples[project].keys()]
            else:     #elif lst[0].startswith('ENSG|hsa|cg')
                fh.write(lst[0]+'\t'+'\t'.join([lst[i+1] for i in sampleIndex])+'\n')
    
    fh.close()
    
    print ('Prepare %s data from project %s is completed !' %(dataFolders[genoDataType], project))

for project in metaProjects.keys():
    limmaInput(project)

Prepare miRNA_Expression_Quantification data from project TCGA-PRAD is completed !
Prepare miRNA_Expression_Quantification data from project TCGA-BLCA is completed !
Prepare miRNA_Expression_Quantification data from project TCGA-LGG is completed !


In [ ]:
### not in use
def changeDirectory(project):
    
    if genoDataType == 1:
        os.chdir(easyTCGA+project+'/'+RNAseq)
    elif genoDataType == 2:
        os.chdir(easyTCGA+project+'/'+miRNA)
    elif genoDataType == 3:
        os.chdir(easyTCGA+project+'/'+methylation)
    elif genoDataType == 4:
        os.chdir(easyTCGA+project+'/'+isoform)
    elif genoDataType == 5:
        os.chdir(easyTCGA+project+'/'+CNS)
    elif genoDataType == 6:
        os.chdir(easyTCGA+project+'/'+maskedCNS)